- date: 2021-04-02 10:17:17
- author: Jerry Su
- slug: Python-Coroutine
- title: Python Coroutine
- category: 
- tags: Python

### eg1

In [1]:
def simple_coroutine():
    print("coroutine started")
    x = yield
    print(f"coroutine received: {x}")

In [2]:
my_coroutine = simple_coroutine()

In [3]:
my_coroutine

<generator object simple_coroutine at 0x7ffa2efdd350>

In [4]:
next(my_coroutine)

coroutine started


In [5]:
my_coroutine.send(42)

coroutine received: 42


StopIteration: 

### eg2

In [1]:
def simple_coroutine2(a):
    print(f"Started: a = {a}")
    b = yield a
    print(f"Received: b = {b}")
    c = yield a + b
    print(f"Received: c = {c}")

In [2]:
my_cor2 = simple_coroutine2(12)

In [3]:
next(my_cor2)

Started: a = 12


12

In [4]:
my_cor2.send(5)

Received: b = 5


17

In [5]:
my_cor2.send(1)

Received: c = 1


StopIteration: 